In [1]:
import pandas as pd
from IPython.core.display import display, HTML
import matplotlib.pyplot as plt
import os
from os.path import join as pjoin
import numpy as np
import pandas as pd
from time import time, sleep, strftime, localtime
from datetime import datetime

%reload_ext autoreload
%autoreload 1

pd.set_option('display.max_rows', 800)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 100)
pd.set_option('display.max_colwidth', 100)

display(HTML("<style>.container { width:100% !important; }</style>"))

import warnings
warnings.filterwarnings('ignore')

In [2]:
import re, string
import requests
from bs4 import BeautifulSoup
from itertools import groupby


In [3]:
main_url = 'http://www.darklyrics.com/'
band_url = 'i/ironmaiden.html'
# band_url = 'm/metallica.html'
album_list_url = main_url + band_url
print(album_list_url)
page = requests.get(album_list_url).text

http://www.darklyrics.com/i/ironmaiden.html


In [4]:
soup = BeautifulSoup(page, "html.parser")

In [5]:
albums = soup.find_all(attrs={"class":"album"})

list_band_albums = list([])

for alb in albums:
    s = alb.find('h2').text
    album_name = re.search('(?<=")(.*)(?=")', s).group(0)
    album_year = re.search('(?<=\()(.*)(?=\))', s).group(0)
    song_names = [i.text for i in alb.find_all('a')]
    song_hrefs = [pjoin(main_url, i['href'][3:]) for i in alb.find_all('a')]
    dict_album = {
        "album":album_name,
        "year":album_year,
        "songs":list(zip(song_names, song_hrefs))
    }
    list_band_albums.append(dict_album)

In [6]:
df_band_albums = pd.DataFrame(list_band_albums)
df_band_albums.head()

,album,songs,year
0,Iron Maiden,"[(Prowler, http://www.darklyrics.com/lyrics/ironmaiden/ironmaiden.html#1), (Sanctuary, http://ww...",1980
1,Running Free,"[(Running Free, http://www.darklyrics.com/lyrics/ironmaiden/runningfree.html#1), (Burning Ambiti...",1980
2,Women In Uniform,"[(Women In Uniform, http://www.darklyrics.com/lyrics/ironmaiden/womeninuniform.html#1), (Invasio...",1980
3,Killers,"[(The Ides Of March, http://www.darklyrics.com/lyrics/ironmaiden/killers.html#1), (Wrathchild, h...",1981
4,The Number Of The Beast,"[(Invaders, http://www.darklyrics.com/lyrics/ironmaiden/thenumberofthebeast.html#1), (Children O...",1982


In [9]:
requests.get(song_href)

<Response [200]>

In [7]:
songs = list([])
song_maps = []
for album in df_band_albums["album"].unique():
    print()
    q = "album==\""+album+"\""
    print(q)
    song_info = df_band_albums.query(q).loc[:,"songs"].values[0]
    for si in song_info:
        sleep(1)
        print(si[0], "\t", si[1])
        song_href = si[1].replace("\\","/")
        print(song_href)
        song_page = requests.get(song_href).text
        song_soup = BeautifulSoup(song_page, "html.parser")
        
        lyrics_section = song_soup.find_all(attrs={"class":"lyrics"})[0]
        lyrics_elem_list = [el for el in lyrics_section if el not in ["\n", "<br/>"]]
        song_headers = lyrics_section.find_all("h3")
        song_headers = [h.text for h in song_headers]
        song_lyrics = [list(g) for k, g in groupby(lyrics_elem_list, key=lambda x: x.name != "h3") if k]
        # Get just the text
        song_lyrics = [[h for h in song if isinstance(h,str)] for song in song_lyrics]
        song_map = {
            "title":[],
            "lyrics":[]
        }
        
        for t,l in list(zip(song_headers, song_lyrics)):
            song_map["title"].append(t)
            song_map["lyrics"].append(l)
        
        song_map = pd.DataFrame(song_map)
        song_map["album"] = album
        song_map["year"] = df_band_albums.query(q).loc[:,"year"].values[0]
        song_maps.append(pd.DataFrame(song_map))
        
        sleep(1)


album=="Iron Maiden"
Prowler 	 http://www.darklyrics.com/lyrics/ironmaiden/ironmaiden.html#1
http://www.darklyrics.com/lyrics/ironmaiden/ironmaiden.html#1


IndexError: list index out of range

In [177]:
song_maps_df = pd.concat(song_maps)
song_maps_df.to_csv("./songs.csv")

''